# Using Twitter API with Tweepy

To interface with Twitter API, we can use third-party package such as Tweepy.
To use the package, we will need to register and get keys from twitter developer portal.  Then, we use these keys to authenticate with OAuth2 to access twitter API.

In [ ]:
import tweepy
import pandas as pd
import pytz
import yaml, os

You can either create a config file 'twitter-config.yml' (in yaml format).  Here is an example of the content in the config file <br>

```yaml
consumer_key: 'XXXXXXXX'
consumer_secret: 'XXXXXXXX'
access_token_key: 'XXXXXXXX'
access_token_secret: 'XXXXXXXX'
```

or change the key/token values in the else section below.

In [ ]:
config_file = 'twitter-config.yml'
if os.path.isfile(config_file):
    config = yaml.safe_load(open(config_file))
else:
    config = {
        'consumer_key': 'XXXXXXXX',
        'consumer_secret': 'XXXXXXXX',
        'access_token_key': 'XXXXXXXX',
        'access_token_secret': 'XXXXXXXX'   
    }

In [ ]:
auth = tweepy.OAuthHandler(config['consumer_key'], config['consumer_secret'])
auth.set_access_token(config['access_token_key'], config['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

Tweepy provides many features:
- searching and listing users' information
- reading tweets from user timelines
- creating, fetching, retweeing tweets
- managing followers
- adding and removing likes
- blocking users
- searching and filtering tweets
- listing trends
- streaming tweets in real-time

Let's create a helper function to print a tweet.  The function prints only one line containing, timestamp (in BKK timezone), user who creates tweet, and the content of the tweet.

In [ ]:
def print_tweet(tweet):
    # tweet.created_at is an "unaware" timezone, but it acutally is a UTC timezone
    # we will have to make it a UTC first, then convert it to bkk
    utc_dt = tweet.created_at.replace(tzinfo=pytz.UTC)
    timezone_bkk = pytz.timezone('Asia/Bangkok')
    bkk_dt = utc_dt.astimezone(timezone_bkk)
    print('{} [{}] {}'.format(bkk_dt, tweet.user.name, tweet.text[:50]))

## Getting user's information
With get_user method, we can access user's public information such as screen_name, description, followers' count, etc.  Please refer to [Python – User object in Tweepy](https://www.geeksforgeeks.org/python-user-object-in-tweepy/) for more details of user object.

In [ ]:
user = api.get_user(screen_name='katyperry')

In [ ]:
print('Name:', user.screen_name)
print('Description:', user.description)
print('Number of Followers:', user.followers_count)
print('Number of Followings:', user.friends_count)
print('Number of Tweets/Retweets:', user.statuses_count)
print('Headshot\'s URL', user.profile_image_url)

## Accessing user timelines
We can get tweets, replies, and mentions in user's timeline, as long as it is public.

In [ ]:
timeline = api.user_timeline(user_id=user.id)
for tweet in timeline:
    print_tweet(tweet)

## Listing Trends
Trends are location-oriented.  We will have to use location coordinate to get trending information.

In [ ]:
# Let's use BKK location
lat = 13.739060668870644
long = 100.53214799610562
location_info = api.closest_trends(lat, long)

In [ ]:
location_info

Get trending keywords and put them in dataframe

In [ ]:
trendings = api.get_place_trends(location_info[0]['woeid'])
df = pd.DataFrame(trendings[0]['trends'])

In [ ]:
# sort trending based on their tweet volume in descending order
trending_df = df.sort_values('tweet_volume', ascending=False)
trending_df

In [ ]:
# get the keyword with the highest tweet volume
top_trending_keyword = trending_df.head(1).iloc[0]['name']
print('Top Trending Keyword = ', top_trending_keyword)

In [ ]:
for tweet in api.search_tweets(q=top_trending_keyword, count=10):
    print_tweet(tweet)

## Understanding cursor
For Twitter API, those information will be returned with some certain limitations e.g. providing only 20 followers.  If you want more complete list, we will have to use Cursor API.  Cursor API is basically a paging mechanism.

In [ ]:
user_id = 'natawutn'
user = api.get_user(screen_name=user_id)

In [ ]:
print('Name:', user.screen_name)
print('Number of Followers:', user.followers_count)
print('Number of Followings:', user.friends_count)
print('Number of Tweets/Retweets:', user.statuses_count)

In [ ]:
for friend in user.friends():
    print('{:15.15} -- {:40.40}'.format(friend.screen_name, friend.description))

In [ ]:
# Iterate through all of the account friends (but not more than 30)
counter = 0
for id in tweepy.Cursor(api.get_friend_ids, screen_name=user_id).items():
    # Process the friend here
    friend = api.get_user(user_id=id)
    print('{:15.15} -- {:40.40}'.format(friend.screen_name, friend.description))
    counter += 1
    if counter >= 30:
        break